# Abstraction Text Mining v1.1
Name: Saad Janjua and Carolyne Gakuria
Date: 9/14/2017


# Import standard packages for Machine Learning

## Imported packages and removed Deprecation Warnings.


In [5]:
#Add packages
# import module(s) into namespace
import pandas as pd   #we almost always need pandas because we like data frames
from bs4 import BeautifulSoup
import requests
import numpy as np
import nltk 
nltk.download()
pd.set_option('display.max_colwidth', 150000) #important for getting all the text
import os
os.getcwd()
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
from sklearn.feature_extraction.text import CountVectorizer
import math
import PyPDF2
from os.path import isfile, join
import sqlite3
import glob
import os
%pwd

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


'C:\\Users\\SJ\\Documents\\GitHub\\BIA 6304\\Project\\Raw'

In [6]:
cd C:\\Users\\SJ\\Documents\\GitHub\\BIA 6304\\Project\\Raw\\

C:\Users\SJ\Documents\GitHub\BIA 6304\Project\Raw


In [7]:
"""If running locally, set these variables to your local directories.
"""
pdf_dir = "C:\\Users\\SJ\\Documents\\GitHub\\BIA 6304\\Project\\Raw\\"
txt_dir = "C:\\Users\\SJ\\Documents\\GitHub\\BIA 6304\\Project\\Processed\\"

In [19]:
"""Note: Uses a generator expression.
Rerun the cell if you restart the loop below.
"""
corpus = (f for f in os.listdir(pdf_dir) if not f.startswith('.') and isfile(join(pdf_dir, f)))


In [20]:
"""The documentation for PyPDF2 is minimal. 
For this pattern, I followed the syntax at 
https://automatetheboringstuff.com/chapter13/ and
https://github.com/msaxton/iliff_review/blob/master/code/atla_pdfConvert.py
"""
for filename in corpus:
    # Open the PDF and load as PyPDF2 Reader object.
    pdf = open(join(pdf_dir, "Test.pdf"),'rb')
    pdfReader = PyPDF2.PdfFileReader(pdf)
    
    # Loop through the pages, extract the text, and write each page to individual file.
    for page in range(0, pdfReader.numPages):
        pageObj = pdfReader.getPage(page)
        text = pageObj.extractText()
        
        # Compile the page name. Add one because Python counts from 0.
        page_name = "{}-pg{}.txt".format(filename[:-4], page+1)

        # Write to each page to file
        with open(join(txt_dir, page_name), mode="w", encoding='utf-8') as o:
            o.write(text)
            
file_list = glob.glob(os.path.join(os.getcwd(), "C:\\Users\\SJ\\Documents\\GitHub\\BIA 6304\\Project\\Processed\\", "*.txt"))

corpus = []

for file_path in file_list:
    with open(file_path) as f_input:
        corpus.append(f_input.read())

df = pd.DataFrame.from_dict(corpus)
df.columns = ['ChartText']
df['Page'] = df.index+1
df.head(5)

,ChartText,Page
0,"ABasicCourseintheTheoryofInterestand\nDerivativesMarkets:\nAPreparationfortheActuarialExamFM/2\nMarcelB.Finan\nArkansasTechUniversity\nc\n\nAllRightsReserved\nPreliminaryDraft\nLastupdated\nMarch24,2017\n",1
1,"10\nTHEBASICSOFINTERESTTHEORY\n1TheMeaningofInterest\nToanalyzetransactions,aclearunderstandingoftheconceptof\ninterestisrequired.Interestcanbeinavarietyofcontexts,suchas\ntheonesfoundindictionariesandencyclopedias.Inthemostcommoncon-\ntext,\ninterest\nisanamountchargedtoaborrowerfortheuseofthelender's\nmoneyoveraperiodoftime.Forexample,ifyouhaveborrowed$100and\nyoupromisedtopayback$105afteroneyearthenthelenderinthiscase\nismakingaof$5,whichisthefeeforborrowinghismoney.Looking\natthisfromthelender'sperspective,themoneythelenderisinvestingis\nchangingvaluewithtimeduetotheinterestbeingadded.Forthatreason,\ninterestissometimesreferredtoasthe\ntimevalueofmoney.\nInterestproblemsgenerallyinvolvefourquantities:principal(s),investment\nperiodlength(s),interestrate(s),amountvalue(s).\nThemoneyinvestedintransactionswillbereferredtoasthe\nprin-\ncipal\n,denotedby\nP:\nTheamountithasgrowntowillbecalledthe\namount\nvalue\nandwillbedenotedby\nA:\nTheerence\nI\n=\nA\n\nP\nisthe\namount\nofinterest\nearnedduringtheperiodofinvestment.Interestexpressedasa\npercentoftheprincipalwillbereferredtoasan\ninterestrate.\nInteresttakesintoaccounttheriskofdefault(riskthattheborrowercan't\npaybacktheloan).Theriskofdefaultcanbereducediftheborrowers\npromisetoreleaseanassetoftheirsintheeventoftheirdefault(theassetis\ncalled\ncollateral\n).\nTheunitinwhichtimeofinvestmentismeasurediscalledthe\nmeasure-\nmentperiod\n.Themostcommonmeasurementperiodisoneyearbutmay\nbelongerorshorter(couldbedays,months,years,decades,etc.).\nExample1.1\nWhichofthefollowingmaytheitionofinterest?\n(a)TheamountIoweonmycreditcard.\n(b)Theamountofcreditremainingonmycreditcard.\n(c)Thecostofborrowingmoneyforsomeperiodoftime.\n(d)Afeechargedonthemoneyyou'veearnedbytheFederalgovernment.\nSolution.\nTheansweris(c)\nExample1.2\nLet\nA\n(\nt\n)denotetheamountvalueofaninvestmentattime\nt\nyears.\n",2
2,"100\nTHEBASICSOFINTERESTTHEORY\nProblem10.15\nz\nTawnymakesadepositintoabankaccountwhichcreditsinterestatanom-\ninalinterestrateof10%perannum,convertiblesemiannually.Atthesame\ntime,Fabiodeposits1,000intoatbankaccount,whichiscredited\nwithsimpleinterest.Attheendof5years,theforcesofinterestonthetwo\naccountsareequal,andFabio'saccounthasaccumulatedto\nZ:\nDetermine\nZ:\nProblem10.16\nFund\nA\nearnsinterestatanominalrateofinterestof12%compounded\nquarterly.Fund\nB\nearnsinterestataforceofinterest\n:\nJohninvested$1,000\nineachfundeyearsago.Today,theamountinFund\nA\nisequaltothe\namountinFund\nB:\nCalculate\n:\nProblem10.17\nAfundearnsinterestataforceofinterestof\n\n=0\n:\n01\nt:\nCalculatethevalue\nattheendof10yearsof8,000investedattheendof5years.\nProblem10.18\nAfundearnsinterestataforceofinterestof\n\n=0\n:\n01\nt:\nCalculatethee\nrateofinterestinthe10thyear.\nProblem10.19\nCalculatethectiveannualinterestrateequivalenttoanominalrateof\ndiscountof6%compoundedcontinuously.\nProblem10.20\nFund\nA\naccumulatesatasimpleinterestrateof10%.Find\nB\naccumulates\natasimplediscountrateof5%.Findthepointintimeatwhichtheforceof\ninterestonthetwofundsareequal.\nProblem10.21\nAninvestmentismadeforoneyearinafundwith\na\n(\nt\n)=\na\n+\nbt\n+\nct\n2\n:\nThenominalrateofinterestearnedduringthehalfoftheyearis5%\nconvertiblesemi-annually.Theerateofinterestearnedfortheentire\nyearis7%.Find\n\n0\n:\n5\n:\nProblem10.22\nIf\na\n(\nt\n)=1+\n:\n01(\nt\n2\n+\nt\n)\n;\ncalculate\n\n5\n:\n",3
3,"10FORCEOFINTEREST:CONTINUOUSCOMPOUNDING\n101\nProblem10.23\nYouaregiventhat\n\n=0\n:\n05\n:\nCalculatetheamountthatmustbeinvestedat\ntheendof10yearstohaveanaccumulatedvalueattheendof30yearsof\n$1,000.\nProblem10.24\nCalculate\nk\nifadepositof1willaccumulateto2.7183in10yearsataforce\nofinterestgivenby:\n\nt\n=\nË†\nkt\n0\n<t\n\n5\n0\n:\n04\nkt\n2\nt>\n5\nProblem10.25\nAdepositismadeonJanuary1,2004.Theinves

In [16]:
# Connect to SQLlite database called A2.db
conn = sqlite3.connect("db_Abstraction.db")
cur = conn.cursor()
database='A2.db'
# Write records stored in a DataFrame to a SQL database as Adult_NEW
df.to_sql("df_NEW", conn, if_exists="replace")
# Check for "?" in DataSet 
pd.read_sql_query("select * from df_NEW;"   , conn)
# Create DF for Adult_NEW
df_NEW = pd.read_sql_query("select * from df_NEW where page = 745;", conn)
df_NEW 

,index,ChartText,Page
0,744,"10FORCEOFINTEREST:CONTINUOUSCOMPOUNDING\n99\nProblem10.8\nz\nAttime\nt\n=0\n;\n1isdepositedintoeachofFund\nX\nandFund\nY:\nFund\nX\naccumulatesataforceofinterest\n\nt\n=\nt\n2\nk\n:\nFund\nY\naccumulatesatanominal\nrateofdiscountof8%perannumconvertiblesemiannually.\nAttime\nt\n=5\n;\ntheaccumulatedvalueofFund\nX\nequalstheaccumulated\nvalueofFund\nY:\nDetermine\nk:\nProblem10.9\nz\nAttime0\n;K\nisdepositedintoFund\nX;\nwhichaccumulatesataforceofinter-\nest\n\nt\n=0\n:\n006\nt\n2\n:\nAttime\nm;\n2\nK\nisdepositedintoFund\nY;\nwhichaccumulates\natanannualeinterestrateof10%.Attime\nn;\nwhere\nn>m;\nthe\naccumulatedvalueofeachfundis4\nK:\nDetermine\nm:\nProblem10.10\nSuppose$2,000isinvestedfor3yearswithaconstantforceofinterestequal\nto9%.\n(a)Findtheaccumulationfunction.\n(b)Findtheerateofinterest.\n(c)Findthevalueoftheinvestmentafter3years.\nProblem10.11\nz\nErniemakesdepositsof100attime0,and\nX\nattime3.Thefundgrowsat\naforceofinterest\n\nt\n=\nt\n2\n100\n;t>\n0\n:\nTheamountofinterestearnedfromtime\n3totime6is\nX:\nCalculate\nX:\nProblem10.12\nIftheforceofinterest\n\nis0.1,obtainthenominalrateofinterest(perannum)\nondepositscompounded(a)monthly;(b)quarterly;(c)annually.\nProblem10.13\nTheforceofinterestdependsupontime,with\n\n(\nt\n)=\n1\n10(1+\nt\n)\n;\n0\n\nt\n\n4\n:\nFindtheaccumulatedamountafterfouryearsforaninitialinvestmentof\n$1,000attimezero.\nProblem10.14\nTheforceofinterest\n\nisconstant.Thecorrespondingnominalrateofdis-\ncountwheninterestiscompoundedp-thlyis\nd\n(\np\n)\n:\nExpress\nd\n(\np\n)\nintermsof\np:\nShowthat\nd\n(\np\n)\nisanincreasingfunctionof\np:\n",745


In [ ]:
# Connect to SQLlite database called A2.db
conn = sqlite3.connect("A2.db")
cur = conn.cursor()
database='A2.db'
# Write records stored in a DataFrame to a SQL database as Adult_NEW
Adult.to_sql("Adult_NEW", conn, if_exists="replace")
# Check for "?" in DataSet 
pd.read_sql_query("select * from Adult_NEW where occupation like '%?%' OR workclass like '%?%';"   , conn)
# Updating rows with "?" for occupation and workclass with Unknown
Values = 'Unknown'
cur.execute("""UPDATE Adult_NEW SET occupation = ? ,workclass = ?,'native-country' = ?""",
  (Values,Values,Values))
conn.commit()
# Updating Target with Y >50K and N <=50K for income
cur.execute("UPDATE Adult_NEW SET Income = 'Yes' WHERE Income like '%>50K%'")
cur.execute("UPDATE Adult_NEW SET Income = 'No' WHERE Income like '%<=50K%'")
conn.commit()
# Create DF for Adult_NEW
Adult_NEW = pd.read_sql_query("select * from Adult_NEW;", conn)
Adult_NEW 
print(Adult_NEW.describe)

# Change working directory

In [2]:
cd O:\\library\\QM\\PHI11\\Dedicated\\Quality_Improvement\\Saad\\SDS Abstraction Study\\Raw\\

O:\library\QM\PHI11\Dedicated\Quality_Improvement\Saad\SDS Abstraction Study\Raw


## Import the Combined Files

In [91]:
#open allows you to read the file
pdfFileObj = open("CombinedFiles.pdf",'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

num_pages = pdfReader.numPages
count = 0
text = ""
#The while loop will read each page
while count < num_pages:
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
#This if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.
    if text != "":
       text = text
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text
    else:
       text = textract.process(fileurl, method='tesseract', language='eng')
# Now we have a text variable which contains all the text derived #from our PDF file. Type print(text) to see what it contains. It #likely contains a lot of spaces, possibly junk such as '\n' etc.
# Now, we will clean our text variable, and return it as a list of keywords.

In [34]:
print(text)

Member
Name:SaadJanjua
DOB:11/13/1990



Hba1CTest
5.8
















Member
Name:JohnJanjua
DOB:11/13/1992




bhzdfbhzd

bzdfr
ndfxbndzxhbzdexhb
nzfgbzfdxbvf
bzxcfvbzd




Hba1CTest
6.0





In [94]:

import glob
import os

file_list = glob.glob(os.path.join(os.getcwd(), "C:\\Users\\SJ\\Documents\\GitHub\\BIA 6304\\Project\\Processed\\", "*.txt"))

corpus = []

for file_path in file_list:
    with open(file_path) as f_input:
        corpus.append(f_input.read())

df = pd.DataFrame.from_dict(corpus)
df.columns = ['ChartText']
df

ValueError: DataFrame constructor not properly called!

# Create a Corpus

In [93]:
#Create a Dataframe
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
# Create a string set
df=StringIO(text)


In [189]:
print(text.count('smoke'))
print(text.count('Hemoglobin'))
print(text.count('Hemoglobin A1C'))
print(text.index('Hemoglobin')) # first occurance

0
0
0


ValueError: substring not found

In [190]:
print(text)

Diagnostic Imaging 3/20/2015 8:55:07 AM PAGE 1/001 Fax Server Electronically Signed By Moretina, Jcnn irer 0312012015 09:40: 53 AM DIAGNOSTIC, IlvtAGING CENTERS, P.A, Edv..r..'vI HBfllliln MO, FACR'jcr,lItler H C/EJ!vey, M Nao, MD, FACR'Nca' K LUlz. MD Llnd<J A H<JfflSOf), MO·eil!nciil A McGhie. MC'$w!l,I Sllo:r, MO' Ja1118j' tl. HiEllmu.;, MD'Gleqory C Reulel. MD Jail!; A. SD"adi()o. Me;' Sld.'r-JY A. Crawe)" MD· Jeffl ey F Bretl/, MD'WIIIf[)m C KOJry, MD DR JENNIFER L MORETINA 2609 GLENN HENDREN DRM: LIBERTY, MO 64068 DATE: 3(2012015 B:43:00AM NAME: JANET HARGARTEN NORTH ole LOCATION DOB: 09/2211961 ACC#: 4857685 DIGITAL MAIoJH,/OGRAlvl -SCREENING BILATERAL -I :-CHNIQUE MLO/CC -leA 0 Secord Look 7. 2-H p:Jstprocessing INDICATION' Screening examinalion. COMPARISON 09/07/2005,01/08/2013,0110912014 MR#: 00058B384 FINDINGS. Th::.c are sCClltered of fibroglandLCir denslly. No suspiCIOUS ,ass, calcification or arc litectural distortion IS IMPRESSION: No mamn-q)raphic of malignancy, 81-RADS

In [98]:
'''
This module converts PDF files from ATLASerials into plain text files. An important part of
this process is striping out the ATLA statment at the end of the PDF. In addition PyPDF tends
to read an - as Š;
'''

import PyPDF2
import re
import os

in_path = "C:\\Users\\SJ\\Documents\\GitHub\\BIA 6304\\Project\\Raw\\"
out_path = "C:\\Users\\SJ\\Documents\\GitHub\\BIA 6304\\Project\\Processed\\"


def make_file_name(fileid):
    file_name = fileid[0:-4]
    return file_name

def clean_text(string_obj):
    clean_text = re.sub("Š", "-", string_obj)
    clean_text = re.sub("¿|¡|£|^|`|«", "", clean_text)
    clean_text = re.sub("Á|Â|Ã|Ä|Å|à|á|â|ã|ä|å", "a", clean_text)
    clean_text = re.sub("È|É|Ê|Ë|è|é|ê|ë", "e", clean_text)
    clean_text = re.sub("Ì|Í|Î|Ï|ì|í|î|ï", "i", clean_text)
    clean_text = re.sub("Ò|Ó|Ô|Õ|Ö|Ø|ò|ó|ô|õ|ö|ø", "o", clean_text)
    clean_text = re.sub("Ù|Ú|Û|Ü|ù|ú|û|ü", "u", clean_text)
    clean_text = re.sub("Ý|ý|ÿ", "y", clean_text)
    return clean_text

def ATLASpdf_to_text(out_path, file_name):
    pdf_pages = []
    pdfObj = open(in_path + fileid, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfObj)
    page_length = pdfReader.numPages

    for i in range(0, (page_length-1)):
        pageObj = pdfReader.getPage(i)
        text = pageObj.extractText()
        pdf_pages.append(text)
    
    pdf_string = ' '.join(pdf_pages) #join with space not new line
    eol_string = "\r\n"

    pdf_string = pdf_string+eol_string # add EOL character to avoid later warnings

    final_string = clean_text(pdf_string)

    with open(out_path + file_name + '.txt', mode='w', encoding='utf-8') as f:
        f.write(final_string)
        
for fileid in os.listdir(in_path):
    if fileid.endswith('.pdf'):
        file_name = make_file_name(fileid)
        ATLASpdf_to_text(out_path, file_name)

        


In [101]:
print(pdf_string)

NameError: name 'pdf_string' is not defined

In [ ]:
##### New Solution

In [192]:
from PyPDF2 import PdfFileReader, PdfFileWriter

with open("CombinedFiles.pdf", 'rb') as infile:

    reader = PdfFileReader(infile)
    writer = PdfFileWriter()
    writer.addPage(reader.getPage(0))

    with open('output.pdf', 'wb') as outfile:
        writer.write(outfile)

In [ ]:
### Solution 3